## Dados sobre as reclamações realizadas no site:

https://www.consumidor.gov.br/pages/principal/?1652964614719

- Dados referentes ao ano de 2022 (Janeiro até Dezembro)
- Dados referentes ao ano de 2023 (Janeiro até Julho)

### Realizando o Pre-Processamento dos Dados

### Bibliotecas Utilizadas:

In [1]:
# Imports

# Manipulaçao dos dados
import pandas as pd
import numpy as np

# Manipulação de arquivos/sistema
import glob
import os

# Ignore Warning(Avisos)
import warnings

warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', 100)

### Carregando a base de dados:

In [2]:
# Unindo os dados dos meses de 2022(Janeiro Até Dezembro) - 2023(Janeiro Até Julho).

# Criando uma variável com a extensão do arquivo a ser lido
extensao = 'csv'

# Agrupando/Concatenando os arqquivos, tudo em um único DataFrame
arquivos_csv = [i for i in glob.glob(f'../Data/Data_for_csv/*.{extensao}')]
df_gov = pd.concat([pd.read_csv(f, sep=';') for f in arquivos_csv])

### Informações do dataset

In [3]:
# Verificando o nome das colunas do DataFrame
df_gov.columns

Index(['Região', 'UF', 'Cidade', 'Sexo', 'Faixa Etária', 'Data Finalização',
       'Tempo Resposta', 'Nome Fantasia', 'Segmento de Mercado', 'Área',
       'Assunto', 'Grupo Problema', 'Problema', 'Como Comprou Contratou',
       'Procurou Empresa', 'Respondida', 'Situação', 'Avaliação Reclamação',
       'Nota do Consumidor'],
      dtype='object')

In [4]:
# Verificando a quantidade de linhas e colunas do DataFrame
df_gov.shape

(2046325, 19)

In [5]:
# Info do dataset
df_gov.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2046325 entries, 0 to 105507
Data columns (total 19 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   Região                  object 
 1   UF                      object 
 2   Cidade                  object 
 3   Sexo                    object 
 4   Faixa Etária            object 
 5   Data Finalização        object 
 6   Tempo Resposta          float64
 7   Nome Fantasia           object 
 8   Segmento de Mercado     object 
 9   Área                    object 
 10  Assunto                 object 
 11  Grupo Problema          object 
 12  Problema                object 
 13  Como Comprou Contratou  object 
 14  Procurou Empresa        object 
 15  Respondida              object 
 16  Situação                object 
 17  Avaliação Reclamação    object 
 18  Nota do Consumidor      float64
dtypes: float64(2), object(17)
memory usage: 312.2+ MB


In [6]:
# Descrição do dataset 
df_gov.describe()

,Tempo Resposta,Nota do Consumidor
count,2.007793e+06,1.024623e+06
mean,6.525656e+00,2.858503e+00
std,3.064824e+00,1.790878e+00
min,0.000000e+00,1.000000e+00
25%,4.000000e+00,1.000000e+00
50%,7.000000e+00,3.000000e+00
75%,9.000000e+00,5.000000e+00
max,1.600000e+01,5.000000e+00


### Verificando valores ausentes

In [7]:
# Calcula o total de células no dataset
totalCells = np.product(df_gov.shape)

# Conta o número de valores ausentes por coluna
missingCount = df_gov.isnull().sum()

# Calcula o total de valores ausentes
totalMissing = missingCount.sum()

# Calcula o percentual de valores ausentes
print("O Dataset tem", round(((totalMissing/totalCells) * 100), 2), "%", "de valores ausentes.")

O Dataset tem 2.73 % de valores ausentes.


In [8]:
# Total de valores ausentes
mis_val = df_gov.isnull().sum()

# Porcentagem de valores ausentes
mis_val_percent = 100 * mis_val / len(df_gov)

# Tipo de dado das colunas com valores ausentes
mis_val_dtype = df_gov.dtypes

# Cria uma tabela com os resultados
mis_val_table = pd.concat([mis_val, mis_val_percent, mis_val_dtype], axis=1)

# Renomear as colunas
mis_val_table_ren_columns = mis_val_table.rename(
columns = {0 : 'Valores Ausentes', 1 : '% de Valores Ausentes', 2: 'Dtype'})

# Classifica a tabela por porcentagem de valores ausentes de forma decrescente e remove colunas sem valores faltantes
mis_val_table_ren_columns = mis_val_table_ren_columns[mis_val_table_ren_columns.iloc[:,0] != 0].sort_values('% de Valores Ausentes', ascending = False).round(2)

# Print 
print ("O dataset tem " + str(df_gov.shape[1]) + " colunas.\n"
        "Encontrado: " + str(mis_val_table_ren_columns.shape[0]) + " colunas que têm valores ausentes.")

# Retorna o dataframe com informações ausentes
mis_val_table_ren_columns

O dataset tem 19 colunas.
Encontrado: 4 colunas que têm valores ausentes.


,Valores Ausentes,% de Valores Ausentes,Dtype
Nota do Consumidor,1021702,49.93,float64
Tempo Resposta,38532,1.88,float64
Sexo,155,0.01,object
Avaliação Reclamação,3,0.00,object


> Vamos adotar o seguinte procedimento para cada coluna:

- Coluna 'Nota do Consumidor' -> Será imputado valores utilizando o método de preenchimento progressivo (forward fill).
- Coluna 'Tempo Resposta' -> será imputado o valor 0(Zero) para os valores NaN - Pois são reclamações que não foram respondidas.
- Coluna 'Sexo' -> será imputado o valor 'O'(Outros) para os valores NaN
- Coluna 'Avaliação Reclamação' -> Dados Ausentes seram removidos

In [ ]:
# Imputando na coluna 'Nota do Consumidor' valores utilizando o método (forward fill)
df_gov['Nota do Consumidor'] = df_gov['Nota do Consumidor'].fillna(method='ffill')

# Imputando na coluna 'Tempo Resposta' o valor 0(Zero) nos valores NaN
df_gov['Tempo Resposta'].fillna(0, inplace=True)

# Imputando na coluna 'Sexo' o Valor O(Outros) nos Valores NaN
df_gov['Sexo'].fillna('O', inplace=True)

# Eliminando valores ausentes na coluna 'Avaliação Reclamação'
df_gov.dropna(inplace=True)

In [ ]:
# Calcula o total de células no dataset
totalCells = np.product(df_gov.shape)

# Conta o número de valores ausentes por coluna
missingCount = df_gov.isnull().sum()

# Calcula o total de valores ausentes
totalMissing = missingCount.sum()

# Calcula o percentual de valores ausentes
print("O Dataset tem", round(((totalMissing/totalCells) * 100), 2), "%", "de valores ausentes.")

> Dataset agora sem valores ausentes

### Verificando/Eliminando linhas duplicadas

In [ ]:
# Conta a quantidade de linhas do dataframe
old = df_gov.shape[0]

# Elimina/apaga as linhas duplicadas
df_gov.drop_duplicates(inplace = True)

# Conta a quantidade de linhas do dataframe (Novo dataframe)
new = df_gov.shape[0]

# Verifica a quandidade de linha duplicadas
count = old - new
if (count == 0):
    print("Nenhuma linha duplicada foi encontrada.")
else:
    print(f"{count} linhas duplicadas foram encontradas e removidas.")

In [ ]:
# Verificando a quantidade de linhas e colunas do DataFrame
df_gov.shape

### Verificando valores únicos

- Coluna 'Região'

In [ ]:
df_gov['Região'].unique()

- Coluna 'UF'

In [ ]:
df_gov['UF'].unique()

- Coluna 'Sexo'

In [ ]:
df_gov['Sexo'].unique()

- Coluna 'Procurou Empresa'

In [ ]:
df_gov['Procurou Empresa'].unique()

- Coluna 'Respondida'

In [ ]:
df_gov['Respondida'].unique()

- Coluna 'Situação'

In [ ]:
df_gov['Situação'].unique()

### Criando novas variáveis

In [ ]:
# Criando a variavel 'Solicitação' aonde cada linha vai receber o numero 1 como identificação
df_gov['Solicitação'] = 1

In [ ]:
# Transformando a coluna 'Data Finalização' de object para datetime
df_gov['Data Finalização'] = pd.to_datetime(df_gov['Data Finalização'])

# Informações do tipo dos dados das colunas
df_gov.info()

In [ ]:
# Criando a variável Mes(Número do mês)
df_gov['Mes Finalização'] = df_gov['Data Finalização'].dt.month

# Criando a variável Mes Nome(Com o nome do mês)
df_gov['Mes Nome Finalização'] = 'NaN'

df_gov.loc[df_gov['Mes Finalização'] == 1, 'Mes Nome Finalização'] = 'Janeiro'
df_gov.loc[df_gov['Mes Finalização'] == 2, 'Mes Nome Finalização'] = 'Fevereiro'
df_gov.loc[df_gov['Mes Finalização'] == 3, 'Mes Nome Finalização'] = 'Março'
df_gov.loc[df_gov['Mes Finalização'] == 4, 'Mes Nome Finalização'] = 'Abril'
df_gov.loc[df_gov['Mes Finalização'] == 5, 'Mes Nome Finalização'] = 'Maio'
df_gov.loc[df_gov['Mes Finalização'] == 6, 'Mes Nome Finalização'] = 'Junho'
df_gov.loc[df_gov['Mes Finalização'] == 7, 'Mes Nome Finalização'] = 'Julho'
df_gov.loc[df_gov['Mes Finalização'] == 8, 'Mes Nome Finalização'] = 'Agosto'
df_gov.loc[df_gov['Mes Finalização'] == 9, 'Mes Nome Finalização'] = 'Setembro'
df_gov.loc[df_gov['Mes Finalização'] == 10, 'Mes Nome Finalização'] = 'Outubro'
df_gov.loc[df_gov['Mes Finalização'] == 11, 'Mes Nome Finalização'] = 'Novembro'
df_gov.loc[df_gov['Mes Finalização'] == 12, 'Mes Nome Finalização'] = 'Dezembro'

# Verificando valore únicos
df_gov['Mes Nome Finalização'].unique()

In [ ]:
# Criando a variável Dia da Semana(Número do Dia da Semana)
df_gov['Dia Semana Finalização'] = df_gov['Data Finalização'].dt.weekday

# Criando a variável Dia Semana Nome Finalização(Com o nome do dia da semana)
df_gov['Dia Semana Nome Finalização'] = 'NaN'

df_gov.loc[df_gov['Dia Semana Finalização'] == 0, 'Dia Semana Nome Finalização'] = 'Segunda-Feira'
df_gov.loc[df_gov['Dia Semana Finalização'] == 1, 'Dia Semana Nome Finalização'] = 'Terça-Feira'
df_gov.loc[df_gov['Dia Semana Finalização'] == 2, 'Dia Semana Nome Finalização'] = 'Quarta-Feira'
df_gov.loc[df_gov['Dia Semana Finalização'] == 3, 'Dia Semana Nome Finalização'] = 'Quinta-Feira'
df_gov.loc[df_gov['Dia Semana Finalização'] == 4, 'Dia Semana Nome Finalização'] = 'Sexta-Feira'
df_gov.loc[df_gov['Dia Semana Finalização'] == 5, 'Dia Semana Nome Finalização'] = 'Sábado'
df_gov.loc[df_gov['Dia Semana Finalização'] == 6, 'Dia Semana Nome Finalização'] = 'Domingo'

# Verificando valore únicos
df_gov['Dia Semana Nome Finalização'].unique()

In [ ]:
# Amostra
df_gov.head()

### Salvando dataset em um arquivo .csv

In [ ]:
# Salvando na pasta 'Data'
df_gov.to_csv('../Data/Data_for_clear/dfGov.csv', sep=';', index=False)
df_gov.to_parquet('../Data/Data_for_clear/dfGov.gzip', index=False)